In [1]:
#loading all single models
import tensorflow as tf
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa
modelv1b0 = load_model('C:\\Users\\Administrator\\Desktop\\v1\\efficientnetv1b0best.h5')
modelv1b1 = load_model('C:\\Users\\Administrator\\Desktop\\v1\\efficientnetv1b1best.h5')
modelv1b2 = load_model('C:\\Users\\Administrator\\Desktop\\v1\\efficientnetv1b2best.h5')
modelv1b3 = load_model('C:\\Users\\Administrator\\Desktop\\v1\\efficientnetv1b3best.h5')
modelv1b4 = load_model('C:\\Users\\Administrator\\Desktop\\v1\\efficientnetv1b4best.h5')
modelv2b0 = load_model('C:\\Users\\Administrator\\Desktop\\v1\\efficientnetv2b0best.h5')
modelv2b1 = load_model('C:\\Users\\Administrator\\Desktop\\v1\\efficientnetv2b1best.h5')
modelv2b2 = load_model('C:\\Users\\Administrator\\Desktop\\v1\\efficientnetv2b2best.h5')
modelv2b3 = load_model('C:\\Users\\Administrator\\Desktop\\v1\\efficientnetv2b3best.h5')
modelv2bS = load_model('C:\\Users\\Administrator\\Desktop\\v1\\efficientnetv2bsbest.h5')


In [2]:
#grad-cam function
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
def grad_cam(input_model, image, layer_name):
    img_arr = tf.keras.preprocessing.image.img_to_array(image)
    x = img_arr[tf.newaxis]
    x = tf.keras.applications.efficientnet_v2.preprocess_input(x)

    grad_model = tf.keras.models.Model([input_model.inputs], [input_model.get_layer(layer_name).output, input_model.output])

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(x)
        class_idx = np.argmax(predictions[0])
        loss = predictions[:, class_idx]


    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]

    gate_f = tf.cast(output > 0, 'float32')
    gate_r = tf.cast(grads > 0, 'float32')

    guided_grads = gate_f * gate_r * grads

    weights = np.mean(guided_grads, axis=(0, 1))
    cam = np.dot(output, weights)

    cam = cv2.resize(cam, (x.shape[1], x.shape[2]), cv2.INTER_LINEAR)

    cam  = np.maximum(cam, 0)

    heatmap = cam / cam.max()


    jet_cam = cv2.applyColorMap(np.uint8(255.0*heatmap), cv2.COLORMAP_JET)

    rgb_cam = np.float32(cv2.cvtColor(jet_cam, cv2.COLOR_BGR2RGB))
    output_arr = cv2.addWeighted(src1=img_arr, alpha=0.7, src2=rgb_cam, beta=0.3, gamma=0)
    output_image = tf.keras.preprocessing.image.array_to_img(output_arr)
    
    return output_image

In [3]:
#function for merging images
def merge_images(file1, file2,file3,file4,file5,file6,file7,file8,file9,file10):
    image1 = file1
    image2 = file2
    image3 = file3
    image4 = file4
    image5 = file5
    image6 = file6
    image7 = file7
    image8 = file8
    image9 = file9
    image10 = file10
    
    (width1, height1) = image1.size
    (width2, height2) = image2.size
    (width3, height3) = image3.size
    (width4, height4) = image4.size
    (width5, height5) = image5.size
    (width6, height6) = image6.size
    (width7, height7) = image7.size
    (width8, height8) = image8.size
    (width9, height9) = image9.size
    (width10, height10) = image10.size
    
    result_width = max(width1 + width2 + width3 + width4 + width5,width6 + width7 + width8 + width9 + width10)
    result_height = max(height1+20, height2+20, height3+20 , height4+20 , height5+20)+ max(height6+20, height7+20, height8+20 , height9+20 , height10+20) 
    v1=max(height1+20, height2+20, height3+20 , height4+20 , height5+20)
    result = Image.new('RGB', (result_width, result_height))
    
    result.paste(im=image1, box=(0, 0))
    result.paste(im=image2, box=(width1, 0))
    result.paste(im=image3, box=(width1+width2, 0))
    result.paste(im=image4, box=(width1+width2+width3, 0))
    result.paste(im=image5, box=(width1+width2+width3+width4, 0))
    
    result.paste(im=image6, box=(0, v1))
    result.paste(im=image7, box=(width6, v1))
    result.paste(im=image8, box=(width6+width7, v1))
    result.paste(im=image9, box=(width6+width7+width8, v1))
    result.paste(im=image10, box=(width6+width7+width8+width9, v1))
    
    return result

In [4]:
#multigradcam for benign
import os
from PIL import Image 
import PIL
from PIL import Image, ImageDraw  
root_src = 'C:\\Users\\Administrator\\Desktop\\1\\test\\benign'

base_input_shapev1b0 = modelv1b0.input_shape[1:]
base_input_shapev1b1 = modelv1b1.input_shape[1:]
base_input_shapev1b2 = modelv1b2.input_shape[1:]
base_input_shapev1b3 = modelv1b3.input_shape[1:]
base_input_shapev1b4 = modelv1b4.input_shape[1:]
base_input_shapev2b0 = modelv2b0.input_shape[1:]
base_input_shapev2b1 = modelv2b1.input_shape[1:]
base_input_shapev2b2 = modelv2b2.input_shape[1:]
base_input_shapev2b3 = modelv2b3.input_shape[1:]
base_input_shapev2bs = modelv2bS.input_shape[1:]

layer_name = 'top_activation'
files=[]
files = os.listdir(root_src)
for file in files:
    #v1
  img_path=os.path.join(root_src,file)
  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev1b0[0], base_input_shapev1b0[1]))
  camv1b0 = grad_cam(modelv1b0, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  v1b0=modelv1b0.predict(x)
  v1b0=v1b0[0][0]



  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev1b1[0], base_input_shapev1b1[1]))
  camv1b1 = grad_cam(modelv1b1, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  v1b1=modelv1b1.predict(x)
  v1b1=v1b1[0][0]

  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev1b2[0], base_input_shapev1b2[1]))
  camv1b2 = grad_cam(modelv1b2, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  v1b2=modelv1b2.predict(x)
  v1b2=v1b2[0][0]



  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev1b3[0], base_input_shapev1b3[1]))
  camv1b3 = grad_cam(modelv1b3, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  v1b3=modelv1b3.predict(x)
  v1b3=v1b3[0][0]

  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev1b4[0], base_input_shapev1b4[1]))
  camv1b4 = grad_cam(modelv1b4, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  v1b4=modelv1b4.predict(x)
  v1b4=v1b4[0][0]
    
    
    
    
    
  #v2  
  img_path=os.path.join(root_src,file)
  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev2b0[0], base_input_shapev2b0[1]))
  camv2b0 = grad_cam(modelv2b0, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet_v2.preprocess_input(x)
  v2b0=modelv2b0.predict(x)
  v2b0=v2b0[0][0]



  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev2b1[0], base_input_shapev2b1[1]))
  camv2b1 = grad_cam(modelv2b1, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet_v2.preprocess_input(x)
  v2b1=modelv2b1.predict(x)
  v2b1=v2b1[0][0]

  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev2b2[0], base_input_shapev2b2[1]))
  camv2b2 = grad_cam(modelv2b2, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet_v2.preprocess_input(x)
  v2b2=modelv2b2.predict(x)
  v2b2=v2b2[0][0]



  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev2b3[0], base_input_shapev2b3[1]))
  camv2b3 = grad_cam(modelv2b3, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet_v2.preprocess_input(x)
  v2b3=modelv2b3.predict(x)
  v2b3=v2b3[0][0]

  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev2bs[0], base_input_shapev2bs[1]))
  camv2bs = grad_cam(modelv2bS, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet_v2.preprocess_input(x)
  v2bs=modelv2bS.predict(x)
  v2bs=v2bs[0][0]
####


  img_pathdes=os.path.join('C:\\Users\\Administrator\\Desktop\\multigradcam\\benign',file)
  result=merge_images(camv1b0,camv1b1,camv1b2,camv1b3,camv1b4,camv2b0,camv2b1,camv2b2,camv2b3,camv2bs)
  d1 = ImageDraw.Draw(result)
  d1.text((5, 370), str(v1b0), fill=(255, 255, 255))
  d1.text((229, 370), str(v1b1), fill=(255, 255, 255))
  d1.text((469, 370), str(v1b2), fill=(255, 255, 255))
  d1.text((729, 370), str(v1b3), fill=(255, 255, 255))
  d1.text((1029, 390), str(v1b4), fill=(255, 255, 255))
  v1bb=(v1b0+v1b1+v1b2+v1b3+v1b4)/5
  d1.text((1100,380 ), str(v1bb), fill=(255, 255, 255))
  
    
  d1.text((5, 770), str(v2b0), fill=(255, 255, 255))
  d1.text((229, 770), str(v2b1), fill=(255, 255, 255))
  d1.text((469, 770), str(v2b2), fill=(255, 255, 255))
  d1.text((729, 770), str(v2b3), fill=(255, 255, 255))
  d1.text((1029, 790), str(v2bs), fill=(255, 255, 255))
  v2bb=(v2b0+v2b1+v2b2+v2b3+v2bs)/5
  d1.text((1100,780 ), str(v2bb), fill=(255, 255, 255))
  result = result.save(img_pathdes)



In [5]:
#multigradcam for malignant
import os
from PIL import Image 
import PIL
from PIL import Image, ImageDraw  
root_src = 'C:\\Users\\Administrator\\Desktop\\1\\test\\malignant'
base_input_shapev1b0 = modelv1b0.input_shape[1:]
base_input_shapev1b1 = modelv1b1.input_shape[1:]
base_input_shapev1b2 = modelv1b2.input_shape[1:]
base_input_shapev1b3 = modelv1b3.input_shape[1:]
base_input_shapev1b4 = modelv1b4.input_shape[1:]
base_input_shapev2b0 = modelv2b0.input_shape[1:]
base_input_shapev2b1 = modelv2b1.input_shape[1:]
base_input_shapev2b2 = modelv2b2.input_shape[1:]
base_input_shapev2b3 = modelv2b3.input_shape[1:]
base_input_shapev2bs = modelv2bS.input_shape[1:]

layer_name = 'top_activation'
files=[]
files = os.listdir(root_src)
for file in files:
    #v1
  img_path=os.path.join(root_src,file)
  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev1b0[0], base_input_shapev1b0[1]))
  camv1b0 = grad_cam(modelv1b0, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  v1b0=modelv1b0.predict(x)
  v1b0=v1b0[0][1]



  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev1b1[0], base_input_shapev1b1[1]))
  camv1b1 = grad_cam(modelv1b1, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  v1b1=modelv1b1.predict(x)
  v1b1=v1b1[0][1]

  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev1b2[0], base_input_shapev1b2[1]))
  camv1b2 = grad_cam(modelv1b2, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  v1b2=modelv1b2.predict(x)
  v1b2=v1b2[0][1]



  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev1b3[0], base_input_shapev1b3[1]))
  camv1b3 = grad_cam(modelv1b3, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  v1b3=modelv1b3.predict(x)
  v1b3=v1b3[0][1]

  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev1b4[0], base_input_shapev1b4[1]))
  camv1b4 = grad_cam(modelv1b4, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  v1b4=modelv1b4.predict(x)
  v1b4=v1b4[0][1]
    
    
    
    
    
  #v2  
  img_path=os.path.join(root_src,file)
  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev2b0[0], base_input_shapev2b0[1]))
  camv2b0 = grad_cam(modelv2b0, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet_v2.preprocess_input(x)
  v2b0=modelv2b0.predict(x)
  v2b0=v2b0[0][1]


  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev2b1[0], base_input_shapev2b1[1]))
  camv2b1 = grad_cam(modelv2b1, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet_v2.preprocess_input(x)
  v2b1=modelv2b1.predict(x)
  v2b1=v2b1[0][1]

  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev2b2[0], base_input_shapev2b2[1]))
  camv2b2 = grad_cam(modelv2b2, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet_v2.preprocess_input(x)
  v2b2=modelv2b2.predict(x)
  v2b2=v2b2[0][1]



  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev2b3[0], base_input_shapev2b3[1]))
  camv2b3 = grad_cam(modelv2b3, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet_v2.preprocess_input(x)
  v2b3=modelv2b3.predict(x)
  v2b3=v2b3[0][1]

  img_pil = tf.keras.preprocessing.image.load_img(img_path, target_size=(base_input_shapev2bs[0], base_input_shapev2bs[1]))
  camv2bs = grad_cam(modelv2bS, img_pil, layer_name)
  img_arr = tf.keras.preprocessing.image.img_to_array(img_pil)
  x = img_arr[tf.newaxis]
  x = tf.keras.applications.efficientnet_v2.preprocess_input(x)
  v2bs=modelv2bS.predict(x)
  v2bs=v2bs[0][1]
####


  img_pathdes=os.path.join('C:\\Users\\Administrator\\Desktop\\multigradcam\\malignant',file)
  result=merge_images(camv1b0,camv1b1,camv1b2,camv1b3,camv1b4,camv2b0,camv2b1,camv2b2,camv2b3,camv2bs)
  d1 = ImageDraw.Draw(result)
  d1.text((5, 370), str(v1b0), fill=(255, 255, 255))
  d1.text((229, 370), str(v1b1), fill=(255, 255, 255))
  d1.text((469, 370), str(v1b2), fill=(255, 255, 255))
  d1.text((729, 370), str(v1b3), fill=(255, 255, 255))
  d1.text((1029, 390), str(v1b4), fill=(255, 255, 255))
  v1bb=(v1b0+v1b1+v1b2+v1b3+v1b4)/5
  d1.text((1100,380 ), str(v1bb), fill=(255, 255, 255))
  
    
  d1.text((5, 770), str(v2b0), fill=(255, 255, 255))
  d1.text((229, 770), str(v2b1), fill=(255, 255, 255))
  d1.text((469, 770), str(v2b2), fill=(255, 255, 255))
  d1.text((729, 770), str(v2b3), fill=(255, 255, 255))
  d1.text((1029, 790), str(v2bs), fill=(255, 255, 255))
  v2bb=(v2b0+v2b1+v2b2+v2b3+v2bs)/5
  d1.text((1100,780 ), str(v2bb), fill=(255, 255, 255))
  #result.show()
  result = result.save(img_pathdes)
#result = result.save(img_pathdes)
